# 作者：Evan_wu        
# Vx号：Evanwu50020

#  学校：NUIST

## $REINFORCE$算法

在强化学习的算法中，有两大类算法，一类是基于策略的，一类是基于价值的，而$REINFOCE$就是典型的基于策略的算法。
那什么叫作基于策略呢，为什么强化学习又要分成这两种算法呢？

简单来说，强化学习的最终目的是让$agent$能够获得足够大的$reward$。那么我们为了让$agent$获得最大的$reward$，我们就要去找到好的策略，因为基于好的策略所选出来的动作是可以得到较大的$reward$的，这就是基于策略的强化学习算法的含义。

而基于价值的强化学习算法则是通过评估从当前状态开始往后有可能得到的$reward$从而得到一个状态价值函数的值$V(s_t)$或者$Q(s_t,a_t)$。（这里的下标t表示的就是当前时刻的状态和当前时刻的动作）接着我们会想办法让我们得到的评估越来越准,并且当我们的评估越来越准之后，我们就会根据评估的值去选择在某一个状态的动作。

我们将一场游戏称为一个回合，将这个游戏里面得到奖励都进行相加，那么得到就是这一场游戏的总奖励，我们定义为$R$。

同时，我们将环境输出的$s$与我们这个$actor$（也就是$agent$）所采取的动作$a$全部都组合起来。那么我们就可以得到下面的这个集合$$\tau =\{s_1,a_1,s_2,a_2......s_t,a_t\}$$
其中字母$\tau$代表是一个完整的游戏回合。


再假设我们现在给定了$actor$（即$agent$）的参数$\theta$,那我们就可以算出发生一个完整的回合的概率：

\begin{equation}
\begin{aligned}
    p_\theta(\tau)    &=p(s_1)p_\theta(a_1|s_1)p(s_2|s_1,a_1)p_\theta(a_2|s_2)p(s_3|s_2,s_2)...\\&=p(s_1)\prod_{t=1}^{T}p_\theta(a_t|s_t)p(s_{t+1} |s_t,a_t)
\end{aligned}
\end{equation}
于此同时，我们定义在某个回合中得到的奖励为$R(\tau)$，这个值就代表了在回合$\tau$中所取得的奖励。但是有一个问题，这个$\tau$虽然是某个回合全部结束后得到的所有奖励，但是这个$\tau$并不可以简单的理解为一个标量，而是应该理解为一个随机变量。

为什么要这么理解？因为当我们去玩一个游戏时，我们会玩非常多的回合，而我们每个回合得到的奖励都会差的很多，那么如果我们只把某一此回合的结果当作这一个游戏带给我们的奖励，就会有非常大的误差。所以我们选择将$R(\tau)$定义为一个随机变量，这个随机变量要用$\tau$这条轨迹出现的概率去乘上这条轨迹返回的奖励来得到，这样我们算出来的就是玩这场游戏会获得的期望奖励，我们将期望奖励定义为$$\overline{R}(\theta)=\sum_{\tau}R(\tau) p_\theta(\tau)$$
其中$p_\theta(\tau)$表示我们玩游戏的时候刚好玩到$\tau$这个回合的概率。那么现在我们的奖励$\overline{R}_\theta$就表示成了一个带参数$\theta$的随机变量，我们想要我们得到的奖励最大，我们就可以对$\overline{R}_\theta$求导，并用梯度上升法求其最大值。

又因为这个式子里只有$p_\theta(\tau)$是和参数$\theta$有关系的，所以我们对$\overline{R}_\theta$求导可以得到下列式子
$$\nabla \overline{R}_\theta = \sum_{\tau}R(\tau)\nabla p_\theta(\tau)$$

得到这个式子之后，我们想办法用$p_\theta(\tau)$对$R(\tau)$进行加权，我们就可以得到下列的式子：
$$\begin{aligned}\nabla \overline{R}_\theta &=\sum_\tau R(\tau)\nabla p_\theta(\tau)\\
                              &=\sum_\tau R(\tau)p_\theta(\tau)\frac{\nabla p_\theta(\tau)}{p_\theta(\tau)}\\
                              &=\sum_\tau R(\tau)p_\theta(\tau)\nabla \log p_\theta(\tau)\\
                              &=E_{\tau \sim p_\theta(\tau)}[R(\tau)\nabla\log p_\theta(\tau)]
                              \end{aligned}$$
                              但我们可以发现这样直接去计算这个期望值是无法计算的，因为我们所采用的概率并不是后面这个式子的采样概率。所以如果我们现在想要得到中括号里这个式子的数学期望，我们就可以采取最笨但也是最没有误差的。我们利用采样的方式对该期望值进行估计：$$\begin{aligned}
                         E_{\tau \sim p_\theta(\tau)}[R(\tau)\nabla\log p_\theta(\tau)]&\approx\frac{1}{N}\sum_{n=1}^{N}R(\tau^{n})\nabla \log p_\theta(\tau^{n})
           =\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T}R(t^{n})\nabla\log p_\theta(a_t^{n}|s_t^{n})
          \end{aligned}$$

关于$\log p_\theta(\tau^{n})$求导的具体推导如下公式：$$\begin{aligned}\nabla\log p_\theta(\tau)
&=\nabla\log p(s_1)+\nabla\sum_{t=1}^{T}\log p_\theta(a_t|s_t)+\nabla\sum_{t=1}^{T}\log p(s_{t+1}|s_t,a_t)\\
&=\sum_{t=1}^{T}\log p_\theta(a_t|s_t)
\end{aligned}$$

所以我们就可以用采样的方式来求得我们奖励的梯度:$$\nabla \overline{R}_\theta=\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T}R(t^{n})\nabla\log p_\theta(a_t^{n}|s_t^{n})$$

接着我们就可以用我们对奖励算出来的梯度更新我们的参数，也就是进行梯度上升。到了这里我们已经完成了最简单的$REINFORCE$的原理讲解，但是我们通常并不会这样去计算我们的梯度，因为这其中有些不合理的地方。

首先，在我们的策略梯度法中，我们希望得到的情况是这样的，如果给定了$s$采取了动作$a$,并且得到了正的奖励，我们就要增加$(s,a)$的概率。如果给定状态$s$,采取了动作$a$,整场游戏得到了负的奖励，我们就要减小在该状态采取$a$的概率。这样看好像没有什么问题，但是如果一些游戏只有正的奖励呢？那么在玩这些只有正值奖励的游戏的时候，如果有一些动作在一整个回合都没有被采样到，这个动作的概率就会下降，因为其它所有采样的动作的概率都会上升，但是这个没有被采样到的动作并不意味着就是一个不好的动作。

所以我们会在我们的奖励上添加一个基线$b$，只有整个回合得到的奖励大于这个基线，该动作的概率才会上升，用公式表示如下
$$\nabla \overline{R}_\theta=\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T}(R(t^{n})-b)\nabla\log p_\theta(a_t^{n}|s_t^{n})$$

第二点，我们在给每个$(s,a)$赋权重的时候是把这一整个回合的奖励都乘了上去，但其实这样是不合理的，因为很有可能所采取的动作明明是一个好的动作，但是因为这个回合所采取的其它动作导致最后整个奖励是一个负值，所以我们就会减少选取这个好的动作的概率。那么为了让奖励变得更合理，我们不再乘整个回合的奖励，我们将当前状态及其之后所得到的奖励作为$(s,a)$的权重，用公式表示如下$$\nabla \overline{R}_\theta=\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T}(\sum_{t^{'}=t}^{T}\gamma^{t^{'}-t} r_{t^{'}}^{n}-b)\nabla\log p_\theta(a_t^{n}|s_t^{n})$$
其中$\gamma^{t^{'}-t} r_{t^{'}}^{n}$表示从当前时刻开始到最后的奖励，$\gamma $表示的是折扣奖励，意味着我们对后期奖励的重视程度，$\gamma=1$代表我们认为长期奖励和当前奖励一样重要，当$\gamma=0$时，我们认为当前奖励要远远比长期奖励重要，并且不考虑长期奖励（一般在实际中我们会将这个值设为0.9）

### 代码实现

In [2]:
import gym
import torch
import torch.nn as nn
import numpy as np
from collections import deque
import torch.optim as optim
from torch.distributions import Categorical
import torch.nn.functional as F

In [9]:
env_name = "LunarLander-v2"
env = gym.make(env_name)
env.seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 64
gamma = 0.99
max_tra = 1000
max_steps = 1000

In [13]:
env.reset()
env.step(0)
env.action_space

Discrete(4)

In [5]:
class actor(nn.Module):
    def __init__(self, s_size,h_size,a_size):
        super(actor,self).__init__()
        self.linear1=nn.Linear(s_size,h_size)
        self.linear2=nn.Linear(h_size,a_size)
    
    def forward(self,x):
        out = self.linear1(x)
        out = self.linear2(out)
        
        return F.softmax(out, dim=1)
    
    def action(self,state):
        state = torch.from_numpy(state).unsqueeze(0).to(device)
        probs = self.forward(state).cpu()

        choice = Categorical(probs)
        action = choice.sample()

        return action.item(), choice.log_prob(action)
    #我们这一步计算的是在当前状态下采取at的可能性的对数值
    

In [14]:
policy = actor(8,hidden_size,4).to(device)
optimizer = optim.Adam(policy.parameters(),lr= 0.001)

In [15]:
def reinforce(env,policy,optimizer,gamma,print_every = 100):
    for i_episode in range(1,max_tra+1):
        state = env.reset()
        episode_reward = []
        probs = []
        for j in range(1,max_steps+1):
            action,prob = policy.action(state)
            next_state,reward,done,_ = env.step(action)
            episode_reward.append(reward);probs.append(prob)
            if done:
                break
        discounts = []#算出折扣奖励
        mean_reward = sum(episode_reward)/len(episode_reward)
        for t in range(0,len(episode_reward)):
            count = 1
            for z in range(t+1,len(episode_reward)):
                episode_reward[t] += episode_reward[z]*gamma** count
                count += 1
            discounts.append(episode_reward[t])
        prob_loss = []
        for a,b in zip(discounts, probs):
            prob_loss.append(-a*b)
        total_prob_loss = torch.cat(prob_loss).sum()
        optimizer.zero_grad()
        total_prob_loss.backward()
        optimizer.step()
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(episode_reward)))
        if np.mean(episode_reward) >= 195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode - 100,
                                                                                       np.mean(episode_reward)))
            break



In [16]:
reinforce(env,policy,optimizer,gamma)

Episode 100	Average Score: -102.43
Episode 200	Average Score: -74.10
Episode 300	Average Score: -128.03
Episode 400	Average Score: -181.83
Episode 500	Average Score: -55.79
Episode 600	Average Score: -57.09
Episode 700	Average Score: -71.72
Episode 800	Average Score: -92.72
Episode 900	Average Score: -190.43
Episode 1000	Average Score: -75.42
